In [1]:
__PRODUCTION__ = 1
__NAME__       = 'motivation'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 1.65

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure, tex
import matplotlib.pyplot as plt

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import dill
import functools
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim

from symfac.experimental import RBFExpansionPlus

In [3]:
def rbf(X, Y=None):
    return np.exp(-np.subtract.outer(X, Y if Y is not None else X)**2)

n = 64

np.random.seed(128)
u = 5 * np.random.randn(n)
v = 4 * np.random.randn(n)
u = gaussian_filter1d(u, 3., order=0)
v = gaussian_filter1d(v, 6., order=0)

K = rbf(u) - rbf(v)
Kt = torch.tensor(K, dtype=torch.float32)

fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].imshow(K)
axs[1].bar(np.arange(n), np.linalg.eigvalsh(K))
plt.show()

U, S, V = np.linalg.svd(K)
svd_K = []
for k in range(1, 10):
    truncation = (U[:, :k] * S[None, :k]) @ V[:k, :]
    svd_K.append(round(float(F.mse_loss(torch.tensor(truncation), torch.tensor(K))), 5))
print(svd_K)

/tmp/ipykernel_1799341/3197144978.py:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


[0.13909, 0.06549, 0.0267, 0.00791, 0.0037, 0.00134, 0.00047, 0.0001, 5e-05]


In [4]:
torch.manual_seed(15513512)

K = rbf(u) - rbf(v)

fac = RBFExpansionPlus(
    k=3,
    batch_size=512,
    max_steps=10001,
)

component_history = []

def component_history_callback(step, model):
    if step in [0, 1, 10, 100, 1000, 10000]:
        component_history.append(dict(
            step=step,
            model=dill.loads(dill.dumps(model))
        ))

fac.fith(
    K.astype(np.float32),
    plugins=[
        dict(
            every=1,
            requires=['step', 'model'],
            callback=component_history_callback
        )
    ],
    u0=0.1 * torch.randn(fac.batch_size, n, fac.k),
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10001/10001 [00:34<00:00, 285.91it/s]


In [5]:
best_run = np.argmin(fac.report.loss_best)

print(fac.optimum.a[best_run, :])

plt.plot(fac.report.loss_history_ticks, fac.report.loss_history[:, best_run])
plt.xscale('log')
plt.yscale('log')
xticks = 2**np.arange(11)
plt.gca().set_xticks(xticks)
plt.gca().set_xticklabels(xticks)
plt.title('Loss of the best run')
plt.show()

fig, axs = plt.subplots(len(component_history), 2, figsize=(10, 4 * len(component_history)))

for snapshot, axrow in zip(component_history, axs):
    step = snapshot['step']
    model = snapshot['model']
    axrow[0].set_title(f'Step {step}')
    im = axrow[0].imshow(
        model(runs=best_run, device='cpu', grad_on=False),
        vmin=-1, vmax=1
    )
    plt.colorbar(mappable=im, ax=axrow[0])
    axrow[1].plot(u, ls=(2, (3, 6)))
    axrow[1].plot(v, ls=(2, (3, 6)))
    axrow[1].plot(model.u[best_run, :, 0].detach().cpu() * fac.optimum.a[best_run, 0].item(), label='comp1')
    axrow[1].plot(model.u[best_run, :, 1].detach().cpu() * fac.optimum.a[best_run, 1].item(), label='comp2')
    axrow[1].plot(model.u[best_run, :, 2].detach().cpu() * fac.optimum.a[best_run, 2].item(), label='comp3')
    axrow[1].legend()
plt.show()

/tmp/ipykernel_1799341/1397850543.py:12: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


tensor([-1.0000e+00, -1.0153e-08,  1.0000e+00], device='cuda:0')


/tmp/ipykernel_1799341/1397850543.py:31: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [6]:
fig = SuperFigure(figsize=(__WIDTH__, __HEIGHT__), dpi=300)
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
x0 = 0.01
dx = 0.25
w = 0.23
axs = [
    fig.make_axes(
        left=x0 + dx * i,
        width=w,
        top=0.2,
        width_to_height=1.0,
        style='modern'
    ) for i in range(4)
]

def svd_reconstruction(k):
    U, S, V = np.linalg.svd(K)
    return torch.tensor((U[:, :k] * S[None, :k]) @ V[:k, :])

image_style = dict(
    cmap='Spectral',
    vmin=-1,
    vmax=1
)
axs[0].imshow(K, **image_style)
axs[1].imshow(svd_reconstruction(1), **image_style)
axs[2].imshow(svd_reconstruction(2), **image_style)
axs[3].imshow(svd_reconstruction(4), **image_style)

title_style = dict(
    fontsize=7,
    y=0.96,
    va='bottom'
)
axs[0].set_title(r'\textbf{Difference of 2 Components}', **title_style)
axs[1].set_title(r'\textbf{Rank-1}', **title_style)
axs[2].set_title(r'\textbf{Rank-2}', **title_style)
axs[3].set_title(r'\textbf{Rank-4}', **title_style)

for ax in axs:
    ax.patch.set_facecolor('w')
    ax.set_xticks([])
    ax.set_yticks([])

ax_group_rbf = fig.make_axes(
    left=axs[0].left, right=axs[0].right, top=0, bottom=1,
    style='blank', zorder=-10
)
ax_group_svd = fig.make_axes(
    left=axs[1].left, right=axs[3].right, top=0, bottom=1,
    style='blank', zorder=-10
)
for ax_group in [ax_group_rbf, ax_group_svd]:
    ax_group.axhline(
        0.9,
        lw=0.5,
        color='k',
    )
group_text_style = dict(
    fontsize=8,
    ha='center',
    va='bottom',
)
ax_group_rbf.text(0.5, 0.92, r'\textbf{RBF Matrix}', **group_text_style)
ax_group_svd.text(0.5, 0.92, r'\textbf{SVD Reconstruction}', **group_text_style)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

/tmp/ipykernel_1799341/1929343541.py:79: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [7]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-motivation.pdf


# Sandbox below

---